- 클래스 변수가 하나의 값에 치우친 데이터, 편향
- 정확도가 높고, 재현율이 낮은 경향

### 용어 정리
- 다수 클래스 : 대부분 샘플이 속한 클래스
- 소수 클래스 : 대부분 샘플이 속하지 않은 클래스
- 위양성 비용(TP) : 부정 샘플을 긍정으로 분류해서 발생하는 비용
- 위음성 비용(TN) : 긍정 샘플을 부정으로 분류해서 발생하는 비용
- 보통은 위음성이 위양성보다 큼
- 절대 부족 : 소수 클래스에 속한 샘플 수가 절대적으로 부족한 상황

### 클래스 불균형 비율
- 불균형 비율이 9 이상이면 편향된 모델이 학습될 가능성이 있음
- 불균형 비율 = 다수 샘플 수 / 소수 샘플 수
- 불균형 비율이 높다고 해서 반드시 편향된 모델을 학습하는 것은 아님

### k-최근접 이웃을 활용
- 클래스 불균형에 매우 민감하므로 진단하는데 적절
- k 값이 클수록 민감, 보통 5 ~ 11
- 불균형 비율 확인 -높음-> k-nn 학습 및 재현율 확인 -낮음->문제있음
                 -낮음-> 문제 없음           -높음-> 문제없음

### 문제 해결의 기본 아이디어
- 소수 클래스에 대한 결정 공간을 넓힘

## 재샘플링
#### 오버샘플링 : 소수 클래스 샘플 생성 -> 원본 데이터가 작을 때 유용
#### 언더샘플링 : 다수 클래스 샘플 삭제 -> 원본 데이터가 클 때 유용
- 결정 경계에 가까운 다수 클래스 샘플 제거
- 결정 경계에 가까운 소수 클래스 샘플 생성 
- 평가 데이터에 대해서는 절대로 재샘플링을 적용하면 안됨

### 대표적인 오버샘플링 알고리즘 : SMOTE
- 대부분의 오버 샘플링 기법이 이 기법에 기반하고 있음
- 소수 클래스 샘플을 임의로 선택하고, 선택된 샘플의 이웃 가운데 하나의 샘플을 또 임의로 선택하여 그 중간에 샘플을 생성하는 과정 반복
- imblearn.over_sampling.SMOTE
- sampling_strategy : 입력하지 않으면 1:1 비율까지 생성, 사전 형태로 입력하여 클래스 별로 샘수 개수 조절 가능
- k_neighbors : SMOTE에서 고려하는 이웃 수 (보통 1,3,5)
- .fit_sample(x,y) : SMOTE을 적용한 결과를 ndarray로 반환

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('./데이터/Secom.csv')
x = df.drop('Y', axis = 1)
y = df['Y']
train_x, test_x, train_y, test_y = train_test_split(x,y)
# 특징이 매우 많음을 확인
train_x.shape

(1175, 590)

In [4]:
# 불균형확인 -> 언더샘플링을 적용하기에 부적적
train_y.value_counts()

-1    1098
 1      77
Name: Y, dtype: int64

In [7]:
# 클래스 불균형 비율 계산
train_y.value_counts().iloc[0] / train_y.value_counts().iloc[-1]

14.25974025974026

In [8]:
# KNN을 사용한 클래스 불균형 테스트
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import *
KNN_model = KNN(n_neighbors = 11).fit(train_x, train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))
# 재현율 불균형이 심각한 수준

0.0
0.9311224489795918


In [14]:
from imblearn.over_sampling import SMOTE
# SMOTE 인스턴스 생성
oversampling_instance = SMOTE(k_neighbors = 3)

# 오버샘플링 적용
o_train_x, o_train_y = oversampling_instance.fit_resample(train_x, train_y)

# ndarray형태가 되므로 다시 DataFrame과 Series로 변환
o_train_x = pd.DataFrame(o_train_x, columns=x.columns)
o_train_y = pd.Series(o_train_y)

In [15]:
# 비율이 1:1인지 확인
o_train_y.value_counts()

-1    1098
 1    1098
Name: Y, dtype: int64

In [17]:
# 같은 모델로 다시 평가 : 정확도는 감소하였으나, 재현율이 크게 오름
KNN_model = KNN(n_neighbors = 11).fit(o_train_x, o_train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))

0.5555555555555556
0.548469387755102


In [18]:
# 정확도를 어느정도 보전한 상태에서 재현율 올리기
oversampling_instance = SMOTE(k_neighbors = 3, sampling_strategy = {1:int(train_y.value_counts().iloc[0]/2),
                             -1:train_y.value_counts().iloc[0]})

# 오버샘플링 적용
o_train_x, o_train_y = oversampling_instance.fit_resample(train_x, train_y)

# ndarray형태가 되므로 다시 DataFrame과 Series로 변환
o_train_x = pd.DataFrame(o_train_x, columns=x.columns)
o_train_y = pd.Series(o_train_y)

In [19]:
KNN_model = KNN(n_neighbors = 11).fit(o_train_x, o_train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))

0.4074074074074074
0.6887755102040817


### 대표적인 언더샘플링 알고리즘 : NearMiss
- 가장 가까운 n개의 소수 클래스 샘플까지 평균 거리가 짧은 다수 클래스 샘플을 순서대로 제거
- imblearn.under_sampling.NearMiss
- sampling_strategy : 입력하지 않으면 1:1 비율까지 생성, 사전 형태로 입력하여 클래스 별로 샘수 개수 조절 가능
- n_neighbors : 평균 거리를 구하는 소수 클래스 샘플 수
- version : NearMiss의 version으로, 2를 설정하면 모든 소수 클래스 샘플까지의 평균 거리를 사용
- .fit_sample(x,y) : NearMiss을 적용한 결과를 ndarray로 반환    

In [21]:
df=pd.read_csv('./데이터/page-blocks0.csv')
x = df.drop('Class', axis = 1)
y = df['Class']
train_x, test_x, train_y, test_y = train_test_split(x,y)
train_y.value_counts()

negative    3677
positive     427
Name: Class, dtype: int64

In [23]:
train_y.replace({'negative': -1, 'positive':1}, inplace = True)
test_y.replace({'negative': -1, 'positive':1}, inplace = True)

# 클래스 불균형 비율 계산
train_y.value_counts().iloc[0] / train_y.value_counts().iloc[-1]

8.611241217798595

In [24]:
# KNN을 사용한 클래스 불균형 테스트
KNN_model = KNN(n_neighbors = 11).fit(train_x, train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))

# 불균형이 심각한 수준은 아님

0.6818181818181818
0.9576023391812866


In [26]:
from imblearn.under_sampling import NearMiss
NM_model = NearMiss(version=2) # version=2 : 모든 소수 클래스 샘플까지의 평균 거리를 활용

u_train_x, u_train_y = NM_model.fit_resample(train_x, train_y)
u_train_x = pd.DataFrame(u_train_x, columns=x.columns)
u_train_y = pd.Series(u_train_y)

In [27]:
u_train_y.value_counts()

-1    427
 1    427
Name: Class, dtype: int64

In [29]:
KNN_model = KNN(n_neighbors = 11).fit(u_train_x, u_train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))

# 정확도가 크게 떨어짐 -> 적당한 비율에 맞게 설정

0.9393939393939394
0.21198830409356725


In [30]:
train_y = NM_model.fit_resample(train_x, train_y)
u_train_x = pd.DataFrame(u_train_x, columns=x.columns)
u_train_y = pd.Series(uNM_model = NearMiss(version=2, sampling_strategy = {1:u_train_y.value_counts().iloc[-1],
                             -1:u_train_y.value_counts().iloc[-1]*5}) # 5:1 비율로

u_train_x, u__train_y)

In [32]:
u_train_y.value_counts()

-1    2135
 1     427
Name: Class, dtype: int64

In [31]:
KNN_model = KNN(n_neighbors = 11).fit(u_train_x, u_train_y)
pred_y = KNN_model.predict(test_x)
print(recall_score(test_y, pred_y))
print(accuracy_score(test_y, pred_y))

0.8181818181818182
0.6652046783625731
